In [24]:
import math
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from sklearn.utils import shuffle

In [51]:
def Sigmoid (a,v):
    temp=-1*a*v
    result=1/(1+math.exp(temp))
    return result


def Tanh (a,v):
    temp=1*a*v
    #result=(1-math.exp(temp))/(1+math.exp(temp))
    result=np.tanh(temp)
    return result



In [41]:
def decode(name):
    if name == 'Iris-setosa':
        return [1,0,0]
    if name == 'Iris-versicolor':
        return [0,1,0]
    else :
        return [0,0,1]

In [42]:
def Initialize_weights(num_layers,num_neuron,bais): 
    if bais ==False :
        first_w=np.random.rand(4,num_neuron[0])
        hidden_w=[]
        hidden_w.append(first_w)
        for i in range(1,num_layers):
            hidden=np.random.rand(num_neuron[i],num_neuron[i-1])
            hidden_w.append(hidden)

    
        ouput_w=np.random.rand(3,num_neuron[num_layers-1])
    
        return hidden_w,ouput_w
    else :
        first_w=np.random.rand(5,num_neuron[0])
        hidden_w=[]
        hidden_w.append(first_w)
        
        for i in range(1,num_layers):
            hidden=np.random.rand(num_neuron[i],num_neuron[i-1]+1)
            hidden_w.append(hidden)
            
        ouput_w=np.random.rand(3,num_neuron[num_layers-1]+1)
        return hidden_w,ouput_w


In [43]:
def predicetion(hidden_w,ouput_w,sample,bais,user_AvtivationFunction,b,num_layers,num_neuron,stander_division):
    x1=sample['X1'].min()
    x2=sample['X2'].min()
    x3=sample['X3'].min()
    x4=sample['X4'].min()
    
    if bais == False :
        ## calc z for hidden
        z=[]     
        for l in range(num_layers):
            if l == 0 :
                temp=[]
                for neuron in range(num_neuron[0]):
                    w=hidden_w[0] ## get weight of first layer and input layer
                    result=x1*w[0,neuron]+x2*w[1,neuron]+x3*w[2,neuron]+x4*w[3,neuron] ##calc net
                    if user_AvtivationFunction== 'Sigmoid':
                        temp.append(Sigmoid(stander_division,result)) ## calc z output of activation function
                    else :
                        temp.append(stander_division*Tanh(b,result))
                #print(temp,'input')
                z.append(temp)

            else :
                temp=[]
                temp_2=z[l-1]  ## get  z of previous layer
                #print(temp_2)
                for neuron in range(num_neuron[l]):
                    w=hidden_w[l] ## array of weight that connected between this and previous layer
                    sum=0
                    for connection in range(num_neuron[l-1]):#### loop previous connections
                        sum=sum+temp_2[connection]*w[neuron,connection]##note col is weights that connect to this neuron

                    if user_AvtivationFunction== 'Sigmoid':
                        temp.append(Sigmoid(stander_division,sum)) ## calc z output of activation function
                    else :
                        temp.append(stander_division*Tanh(b,sum))

                z.append(temp)



        ## calc y for output
        y=[]
        temp_2=z[-1]  ## get  z of last layer
        for output in range(3):
            w=ouput_w
            sum=0
            for connection in range(num_neuron[-1]):
                sum=sum+temp_2[connection]*w[output,connection]

            if user_AvtivationFunction== 'Sigmoid':
                y.append(Sigmoid(stander_division,sum))
            else :
                y.append(stander_division*Tanh(b,sum))
   
    ##################################### with bais ########################
    else :
        ## calc z for hidden
        z=[] 

        for l in range(num_layers):

            if l == 0 :
                temp=[]
                for neuron in range(num_neuron[0]):
                    w=hidden_w[0] ## get weight of first layer and input layer
                    result=w[0,neuron]+x1*w[1,neuron]+x2*w[2,neuron]+x3*w[3,neuron]+x4*w[4,neuron] ##calc net
                    if user_AvtivationFunction== 'Sigmoid':
                        temp.append(Sigmoid(stander_division,result)) ## calc z output of activation function
                    else :
                        temp.append(stander_division*Tanh(b,result))
                        
                z.append(temp)

            else :
                temp=[]
                temp_2=z[l-1]  ## get  z of previous layer
                for neuron in range(num_neuron[l]):
                    w=hidden_w[l] ## array of weight that connected between this and previous layer

                    sum=0
                    for connection in range(num_neuron[l-1]+1):#### loop previous connections
                        if connection == 0 :
                            sum=sum+w[neuron,connection]
                        else :
                            sum=sum+temp_2[connection-1]*w[neuron,connection]##note col is weights that connect to this neuron

                    if user_AvtivationFunction== 'Sigmoid':
                        temp.append(Sigmoid(stander_division,sum)) ## calc z output of activation function
                    else :
                        temp.append(stander_division*Tanh(b,sum))

                z.append(temp)

        ## calc y for output
        y=[]
        temp_2=z[-1]  ## get  z of last layer

        for output in range(3):
            w=ouput_w
            sum=0
            for connection in range(num_neuron[-1]+1):
                if connection == 0 :
                    sum=sum+w[output,connection]
                else :
                    sum=sum+temp_2[connection-1]*w[output,connection]
            if user_AvtivationFunction== 'Sigmoid':
                y.append(Sigmoid(stander_division,sum))
            else :
                y.append(stander_division*Tanh(b,sum))


    #print(y)
    if y[0]>y[1] :
        if y[0]>y[2]:
            return [1,0,0]
        else :
            return [0,0,1]
    else :
        if y[1]>y[2]:
            return [0,1,0]
        else :
            return [0,0,1]


In [44]:
def back_propagation(stander_division,epoches,train_data,b,learning_rate,hidden_w,
                     ouput_w,bais,num_layers,num_neuron,user_AvtivationFunction):
    if bais == False :
        for epoch in range(epoches):
            for i in range(int(train_data.size/5)):## should loop at 90 
                print(epoch,end = "\r")
                ## get row from iris
                samples=train_data[i:i+1]
                
                x1=samples['X1'].min()
                x2=samples['X2'].min()
                x3=samples['X3'].min()
                x4=samples['X4'].min()
                target=decode(samples['Class'].min())
                
                ##################################### feedforward 1 #####################################

                ## calc z for hidden
                z=[] 

                for l in range(num_layers):
                    if l == 0 :
                        temp=[]
                        for neuron in range(num_neuron[0]):
                            w=hidden_w[0] ## get weight of first layer and input layer
                            result=x1*w[0,neuron]+x2*w[1,neuron]+x3*w[2,neuron]+x4*w[3,neuron] ##calc net
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append(Sigmoid(stander_division,result)) ## calc z output of activation function
                            else :
                                temp.append(stander_division*Tanh(b,result))
                        z.append(temp)

                    else :
                        temp=[]
                        temp_2=z[l-1]  ## get  z of previous layer
                        for neuron in range(num_neuron[l]):
                            w=hidden_w[l] ## array of weight that connected between this and previous layer

                            sum=0
                            for connection in range(num_neuron[l-1]):#### loop previous connections
                                sum=sum+temp_2[connection]*w[neuron,connection]##note col is weights that connect to this neuron

                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append(Sigmoid(stander_division,sum)) ## calc z output of activation function
                            else :
                                temp.append(stander_division*Tanh(b,sum))
                                
                        z.append(temp)
                        
                #print('z',z)        
                ## calc y for output
                y=[]
                temp_2=z[-1]  ## get  z of last layer
                for output in range(3):
                    w=ouput_w

                    sum=0
                    for connection in range(num_neuron[-1]):
                        sum=sum+temp_2[connection]*w[output,connection]
                        
                        
                    if user_AvtivationFunction== 'Sigmoid':
                        y.append(Sigmoid(stander_division,sum))
                    else :
                        y.append(stander_division*Tanh(b,sum))
                    

                
                ##################################### back-propagation  #####################################


                ## calc sk for output
                sk=[]
                #print('t',target)
                #print('y',y)
                for j in range(3):
                    if user_AvtivationFunction== 'Sigmoid':
                        sk.append(  stander_division*(target[j]-y[j])*y[j]*(1-y[j]) )  ## for segmoid function
                    else :
                        sk.append( ( b/stander_division)*(target[j]-y[j])*(stander_division-y[j])
                                                       *(stander_division+y[j]))

                #print('sk',sk)
                ## calc sj for hidden layers
                sj=[]
                for layer in range(num_layers,0,-1): ## calc sj for each neurons in hidden layers

                    if layer == num_layers : ## layer connent to output 
                        temp=[]
                        temp_2=z[num_layers-1]
                        for neuron in range(num_neuron[num_layers-1]):#calc sj for each neuron in last layer
                            zj=temp_2[neuron]

                            w=ouput_w
                            sum=0
                            for conn in range(3):
                                sum=sum+w[conn,neuron]*sk[conn]
                            #print('sum',sum)
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append( stander_division*zj*(1-zj)*sum ) ## sigmoid activation
                            else :
                                temp.append((b/stander_division)*(stander_division-zj)
                                            *(stander_division+zj)* sum) ## tanh activation
                            
                        sj.append(temp)
                        
                        #print('sj',sj)

                    else :
                        temp=[]
                        temp_2=z[layer-1]
                        temp_3=sj[num_layers-layer-1]
                        for neuron in range(num_neuron[layer-1]):#calc sj for each neuron in layer
                            zj=temp_2[neuron]

                            w=hidden_w[layer]
                            sum=0
                            for conn in range(num_neuron[layer]):
                                sum=sum+w[conn,neuron]*temp_3[conn]
                                
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append( stander_division*zj*(1-zj)*sum ) ## sigmoid activation
                            else :
                                temp.append((b/stander_division)*(stander_division-zj)
                                            *(stander_division+zj)* sum) ## tanh activation

                        sj.append(temp)
                        
                #print('sj',sj)
                ##################################### feedforward (updata weights) 2  #########################
                for layer in range(num_layers+1): ##loop at every layer
                    if layer == num_layers:
                        w=ouput_w
                        temp_2=z[num_layers-1]
                        for neuron in range(num_neuron[num_layers-1]):# for each neuron in last layer

                            for conn in range(3):
                                ouput_w[conn,neuron]= w[conn,neuron]+learning_rate*sk[conn]*temp_2[neuron]
                                #ouput_w[conn,neuron]=1
                        
                    
                    elif layer == 0 :
                        #print('>>>>>>>>')
                        #print(hidden_w[layer])
                        w=hidden_w[layer]
                        temp=sj[num_layers-layer-1] ## because sj in order inverse 
                        #print(temp)
                        for neuron in range(num_neuron[layer]):# for each neuron in last layer 
                            w[0,neuron]=w[0,neuron]+learning_rate*temp[neuron]*x1
                            w[1,neuron]=w[1,neuron]+learning_rate*temp[neuron]*x2
                            w[2,neuron]=w[2,neuron]+learning_rate*temp[neuron]*x3
                            w[3,neuron]=w[3,neuron]+learning_rate*temp[neuron]*x4
                        hidden_w[layer]=w
                        #print('<<<<<<')
                        #print(w)
                        
                    else :
                        w=hidden_w[layer]
                        temp_2=z[layer-1]
                        temp=sj[num_layers-layer-1]
                        #print(temp," sj ")
                        #print(temp_2," z ")
                        for neuron in range(num_neuron[layer]):# for each neuron in hidden layer

                            for conn in range(num_neuron[layer-1]):
                                w[neuron,conn]=w[neuron,conn]+learning_rate*temp[neuron]*temp_2[conn]
                        hidden_w[layer]=w
                        
    else:  ############ with bais
        for epoch in range(epoches):
            for i in range(int(train_data.size/5)):## should loop at 90
                print(epoch,end = "\r")
                ## get row from iris
                samples=train_data[i:i+1]

                x1=samples['X1'].min()
                x2=samples['X2'].min()
                x3=samples['X3'].min()
                x4=samples['X4'].min()
                target=decode(samples['Class'].min())

                ##################################### feedforward 1 #####################################

                ## calc z for hidden
                z=[] 

                for l in range(num_layers):
                    
                    if l == 0 :
                        temp=[]
                        for neuron in range(num_neuron[0]):
                            w=hidden_w[0] ## get weight of first layer and input layer
                            result=w[0,neuron]+x1*w[1,neuron]+x2*w[2,neuron]+x3*w[3,neuron]+x4*w[4,neuron] ##calc net
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append(Sigmoid(stander_division,result)) ## calc z output of activation function
                            else :
                                temp.append(stander_division*Tanh(b,result))
                            
                        z.append(temp)

                    else :
                        temp=[]
                        temp_2=z[l-1]  ## get  z of previous layer
                        for neuron in range(num_neuron[l]):
                            w=hidden_w[l] ## array of weight that connected between this and previous layer

                            sum=0
                            for connection in range(num_neuron[l-1]+1):#### loop previous connections
                                if connection == 0 :
                                    sum=sum+w[neuron,connection]
                                else :
                                    sum=sum+temp_2[connection-1]*w[neuron,connection]##note col is weights that connect to this neuron

                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append(Sigmoid(stander_division,sum)) ## calc z output of activation function
                            else :
                                temp.append(stander_division*Tanh(b,sum))

                        z.append(temp)

                ## calc y for output
                y=[]
                temp_2=z[-1]  ## get  z of last layer
                
                for output in range(3):
                    w=ouput_w
                    sum=0
                    for connection in range(num_neuron[-1]+1):
                        if connection == 0 :
                            sum=sum+w[output,connection]
                        else :
                            sum=sum+temp_2[connection-1]*w[output,connection]
                            
                    if user_AvtivationFunction== 'Sigmoid':
                        y.append(Sigmoid(stander_division,sum))
                    else :
                        y.append(stander_division*Tanh(b,sum))

                
                ##################################### back-propagation  #####################################


                ## calc sk for output
                sk=[]

                for j in range(3):
                    if user_AvtivationFunction== 'Sigmoid':
                        sk.append(  stander_division*(target[j]-y[j])*y[j]*(1-y[j]) )  ## for segmoid function
                    else :
                        sk.append( (b/stander_division)*(target[j]-y[j])*(stander_division-y[j])
                                                       *(stander_division+y[j]))


                ## calc sj for hidden layers
                sj=[]
                for layer in range(num_layers,0,-1): ## calc sj for each neurons in last hidden layers

                    if layer == num_layers : ## layer connent to output 
                        temp=[]
                        temp_2=z[num_layers-1]
                        for neuron in range(num_neuron[num_layers-1]):#calc sj for each neuron in last layer
                            zj=temp_2[neuron]
                            w=ouput_w
                            sum=0                            
                            for conn in range(3):
                                sum=sum+w[conn,neuron+1]*sk[conn]
       
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append( stander_division*zj*(1-zj)*sum ) ## sigmoid activation
                            else :
                                temp.append((b/stander_division)*(stander_division-zj)
                                            *(stander_division+zj)* sum) ## tanh activation
                            
                        sj.append(temp)

                    else :
                        temp=[]
                        temp_2=z[layer-1]
                        temp_3=sj[num_layers-layer-1]
                        for neuron in range(num_neuron[layer-1]):#calc sj for each neuron in layer
                            zj=temp_2[neuron]

                            w=hidden_w[layer]
                            
                            sum=0
                            for conn in range(num_neuron[layer]):
                                sum=sum+w[conn,neuron]*temp_3[conn]
                                
                            if user_AvtivationFunction== 'Sigmoid':
                                temp.append( stander_division*zj*(1-zj)*sum ) ## sigmoid activation
                            else :
                                temp.append((b/stander_division)*(stander_division-zj)
                                            *(stander_division+zj)* sum) ## tanh activation

                        sj.append(temp)

                ##################################### feedforward (updata weights) 2  #########################
                for layer in range(num_layers+1): ##loop at every layer
                    if layer == num_layers:
                        w=ouput_w
                        temp_2=z[num_layers-1]
                        for neuron in range(num_neuron[num_layers-1]):# for each neuron in last layer
                            
                            for conn in range(3):
                                if neuron == 0:
                                    ouput_w[conn,neuron]= w[conn,neuron]+learning_rate*sk[conn]*1
                                else :
                                    ouput_w[conn,neuron]= w[conn,neuron]+learning_rate*sk[conn]*temp_2[neuron]


                    elif layer == 0 :
                        w=hidden_w[layer]
                        temp=sj[num_layers-layer-1]  ## because sj in order inverse 
                        for neuron in range(num_neuron[layer]):# for each neuron in last layer
                            w[0,neuron]=w[0,neuron]+learning_rate*temp[neuron]*1
                            w[1,neuron]=w[1,neuron]+learning_rate*temp[neuron]*x1
                            w[2,neuron]=w[2,neuron]+learning_rate*temp[neuron]*x2
                            w[3,neuron]=w[3,neuron]+learning_rate*temp[neuron]*x3
                            w[4,neuron]=w[4,neuron]+learning_rate*temp[neuron]*x4
                        hidden_w[layer]=w

                    else :
                        w=hidden_w[layer]
                        temp_2=z[layer-1]
                        temp=sj[num_layers-layer-1]
                        #print(temp," sj ")
                        #print(temp_2," z ")
                        #print(w)
                        for neuron in range(num_neuron[layer]):# for each neuron in hidden layer

                            for conn in range(num_neuron[layer-1]):
                                #print(conn)
                                if conn==0 :
                                    w[neuron,conn]=w[neuron,conn]+learning_rate*temp[neuron]*1
                                else :
                                    w[neuron,conn]=w[neuron,conn]+learning_rate*temp[neuron]*temp_2[conn]
                                    
                        hidden_w[layer]=w
            
    return hidden_w,ouput_w         

In [45]:
def CreateConfuionMatrix(actual_output, prediction):
    """
    Purpose: Create Confusion Matrix and Print output status
    :param actual_output: the real values of tested data
    :param prediction: the calculated output of trained parameters
    :return: accuracy
    """
    # Defined Variables
    columns = ['actual', 'prediction', 'status']
    row = list()
    test_status = pd.DataFrame(columns=columns)
    confusion_matrix = np.zeros([2, 2])

    # Creating Confusion Matrix Process and Get status of output
    for i in range(len(prediction)):
        if actual_output[i] == 1:
            if prediction[i] == 1:
                confusion_matrix[0][0] += 1
                row.append([actual_output[i], prediction[i], 'Matching'])
            else:
                confusion_matrix[0][1] += 1
                row.append([actual_output[i], prediction[i], 'Mismatching'])

        elif actual_output[i] == -1:
            if prediction[i] == -1:
                confusion_matrix[1][1] += 1
                row.append([actual_output[i], prediction[i], 'Matching'])
            else:
                confusion_matrix[1][0] += 1
                row.append([actual_output[i], prediction[i], 'Mismatching'])

    # Then Print status of each output
    rows = pd.DataFrame(row, columns=columns)
    test_status = test_status.append(rows, ignore_index=True)
    print("--> The Test is Now Running ...")
    print(test_status)

    # Calculate accuracy by [sum of diagonal / total sum]
    accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)

    # Show Confusion Matrix
    print("--> Confusion Matrix:")
    print(confusion_matrix)

    # return the accuracy
    return accuracy

In [52]:
def CreateForm(size, title):
    form = Tk()
    form.geometry(size)
    form.title(title)
    form.resizable(False, False)
    return form

def CreateLabel(text):
    lbl_var = StringVar()
    lbl_var.set(text)
    lbl_features = Label(master=master, textvariable=lbl_var, font=fontStyle)
    lbl_features.pack(fill='x', padx=5, pady=5)

def CreateComboBox(data):
    cmb_obj = ttk.Combobox(master=master, textvariable=StringVar(), font=fontStyle)
    cmb_obj['values'] = data
    cmb_obj['state'] = 'readonly'
    cmb_obj.pack(fill='x', padx=5, pady=5)
    return cmb_obj

def CreateTextbox():
    txt_entry = Entry(master, font=fontStyle)
    txt_entry.pack(fill='x', padx=5, pady=5)
    return txt_entry

def CreateCheckbox(text):
    cb_var = IntVar()
    cb_obj = Checkbutton(master=master, text=text, variable=cb_var, onvalue=1, offvalue=0, font=fontStyle)
    cb_obj.pack()
    return cb_var

def GetSelectedAvtivatioFunction():
    selected_option = classes_cmb.get()
    return selected_option

def GetLearningRate():
    return float(txt_LearningRate.get())

def GetNumberOfHiddenLayer():
    return int(txt_Number_Hidden_Layer.get())

def GetNumberOfNeuronPerHiddenLayer():
    NNPH = txt_Num_Neurons_Per_HiddenLayer.get()
    if ',' in NNPH:
        num = [int(x)for x in NNPH.split(',')]
    else:
        num = int(NNPH)
    return num

def GetEpochNumber():
    return int(txt_Epochs.get())


def GetBiasDesicion():
    return int(use_bias.get())

def RunWholeProgram():
    ###### reading data #####
    dataset = pd.read_csv('E:\\IrisData.txt')

    class_a, class_b, class_c = dataset[:50], dataset[50:100], dataset[100:150]

    class_a_train, class_b_train, class_c_train = class_a[:30], class_b[:30], class_c[:30]
    class_a_test, class_b_test, class_c_test = class_a[30:], class_b[30:], class_c[30:]

    train_data = pd.concat([class_a_train, class_b_train, class_c_train])
    test_data = pd.concat([class_a_test, class_b_test, class_c_test])

    train_data = train_data.sample(frac=1)
    train_data=shuffle(train_data)
    test_data = test_data.sample(frac=1)
    ########################
    
    # Defined Variables
    
    # Get Results From Form
    user_AvtivationFunction = GetSelectedAvtivatioFunction()
    user_lr = GetLearningRate()
    user_epochs = GetEpochNumber()
    user_bias = GetBiasDesicion()
    numberOfHiddenLayers = GetNumberOfHiddenLayer()
    numberOfNeuronsPerHiddenLayer = GetNumberOfNeuronPerHiddenLayer()  
    
    
    #####  choice ##########
    if user_bias == 1 :
        bais=True
    else :
        bais=False
       
    activation_choice=user_AvtivationFunction  ## 
    ########################

    ###### Initialize the weights #####
    num_layers=numberOfHiddenLayers
    num_neuron=numberOfNeuronsPerHiddenLayer
    
    ###hyper_parameter ###
    stander_division=.5#need for sigmoid
    epoches=user_epochs
    b=.3
    learning_rate=user_lr
    ######################
    
    
    
    
    hidden_w,ouput_w=Initialize_weights(num_layers,num_neuron,bais)
    
    #print(hidden_w,ouput_w)
    model_W_hidden,model_w_output=back_propagation(stander_division,epoches,train_data,b,learning_rate,hidden_w,
                                        ouput_w,bais,num_layers,num_neuron,user_AvtivationFunction)
    
    #print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    #print(model_W_hidden,model_w_output)
    
    
    s=0
    for i in range((int(train_data.size/5))):
        sample=train_data[i:i+1]
        target=decode(sample['Class'].min())
        y=predicetion(model_W_hidden,model_w_output,sample,bais,
                          user_AvtivationFunction,b,num_layers,num_neuron,stander_division)
        if y == target :
            s=s+1
        
    train_accuracy=(s/(int(train_data.size/5)))
    #print('Train accuracy',train_accuracy )  
        
    sum=0
    actual_output=[]
    prediction_value=[]
    for i in range((int(test_data.size/5))):#(int(test_data.size/5))
        sample=test_data[i:i+1]

        target=decode(sample['Class'].min())
        actual_output.append(target)
        y=predicetion(model_W_hidden,model_w_output,sample,bais,
                      user_AvtivationFunction,b,num_layers,num_neuron,stander_division)
        prediction_value.append(y)
        #print(y)   
        if y == target :
            sum=sum+1
         
    test_accuracy=(sum/(int(test_data.size/5))) 
    #print('test accuracy',test_accuracy)
    #accuracy=CreateConfuionMatrix(actual_output, prediction_value)
    #print(accuracy)
    print('Avtivation',' ','LR',' ','Epochs','Layer','HiddenNodes' )
    print(' ',user_AvtivationFunction,'  ',learning_rate,
          '   ',user_epochs,'   ',numberOfHiddenLayers,'   ',num_neuron)
    print()
    print()
    print('Train accuracy','     ','Test accuracy')
    print(train_accuracy,'   ',test_accuracy)
####################### Start of Main ###########################
# Create main form
master = CreateForm(size="350x450", title="TASK 2 SOLUTION")
fontStyle = tkFont.Font(family="JetBrains Mono", size=10)


# Create Label and Combobox for Classes
classes = ('Sigmoid', 'Hyperbolic')
CreateLabel(text="Select activation function")
classes_cmb = CreateComboBox(data=classes)

# Create Label and TextBox for numberOfHiddenLayer
CreateLabel(text="Enter number of hidden layers")
txt_Number_Hidden_Layer = CreateTextbox()

# Create Label and TextBox for N of Neuron per hidden Layer
CreateLabel(text="Enter number of neurons per hidden layer")
txt_Num_Neurons_Per_HiddenLayer = CreateTextbox()

# Create Label and TextBox for Learning Rate
CreateLabel(text="Enter Learning rate Value")
txt_LearningRate = CreateTextbox()

# Create Label and TextBox for Epochs
CreateLabel(text="Enter Epochs Number")
txt_Epochs = CreateTextbox()

# Create Label and Checkbox for bias
CreateLabel(text="Please Check this for Selecting Bias")
use_bias = CreateCheckbox(text="Use Bias")

# Create Button
btn_submit = Button(master=master, text="Submit", width=15, command=RunWholeProgram, font=fontStyle)
btn_submit.pack(pady=15)

# Run the Form
master.mainloop()

Avtivation   LR   Epochs Layer HiddenNodes
  Hyperbolic    0.4     1000     2     [7, 4]


Train accuracy       Test accuracy
0.6666666666666666     0.6666666666666666
